# Set Your Data Free with Kafka - Java

In [ ]:
USE GamePlayDB;

In [ ]:
SELECT DB_NAME();

## Create Kafka Library

Replace the `<full-path-to>` below with the path to the `SqlToKafka-1.0.jar` file. You can find it in the download under `..\code\bin\java\`.

In [ ]:
CREATE EXTERNAL LIBRARY javePublishToKafka 
FROM (CONTENT = '<full-path-to>\SqlToKafka-1.0.jar')
WITH (LANGUAGE = 'Java');

In [ ]:
SELECT * FROM sys.external_libraries;

## Test the code

Before running the code ensure the JRE_HOME variable is set to where the AZUL-OpenJDK-JRE is.

In [ ]:

EXEC sp_execute_external_script @language = N'Java',
                                  @script = N'kafkapublish.SqlKafka',
                                  @input_data_1 = N'SELECT kc.BootstrapServers, ''1'', kes.Topic
                                                    FROM dbo.tb_KafkaCluster kc
                                                    JOIN dbo.tb_KafkaEventSubscriber kes
                                                      ON kc.ClusterID = kes.ClusterID
                                                    WHERE kes.EventID =  1 ',
                                  @params = N'@msg nvarchar(max), @eventID int, @partVal nvarchar(50)',
                                  @eventID = 1,
                                  @partVal = '1',
                                  @msg = 'Hello World Java';

## Hook Procs

In [ ]:
CREATE OR ALTER PROCEDURE dbo.pr_PublishToKafkaJava @EventID int,
                                                @PartitionValue nvarchar(50),
                                                @EventMessage nvarchar(max)
AS
BEGIN

  IF(@PartitionValue = '')
  BEGIN
    SET @PartitionValue = NULL
  END

  EXEC sp_execute_external_script @language = N'Java',
                                  @script = N'kafkapublish.SqlKafka',
                                  @input_data_1 = N'SELECT kc.BootstrapServers, @partVal, kes.Topic
                                                    FROM dbo.tb_KafkaCluster kc
                                                    JOIN dbo.tb_KafkaEventSubscriber kes
                                                      ON kc.ClusterID = kes.ClusterID
                                                    WHERE kes.EventID =  @eventID ',
                                  @params = N'@msg nvarchar(max), @eventID int, @partVal nvarchar(50)',
                                  @eventID = @EventID,
                                  @partVal = @PartitionValue,
                                  @msg = @EventMessage;
END

### Test the hook proc

In [ ]:
EXEC dbo.pr_PublishToKafkaJava @EventID = 1,
                              @PartitionValue = '2',
                              @EventMessage = 'Hello World Java4'

### Hook it into the generate proc

In [ ]:
CREATE OR ALTER PROCEDURE dbo.pr_GenerateAndPublishWagerEvent @UserID INT,
                                                              @GameID INT,
                                                              @WagerAmount bigint,
                                                              @PayoutAmount bigint,
                                                              @EventTime datetime2,
                                                              @eventID int
AS
BEGIN
  DECLARE @msg nvarchar(max);
 
   --generate the event
  SET @msg =  (SELECT 1500 AS eventTypeId,
                 @UserId AS userId,
                 @GameId AS gameId,
                 @WagerAmount AS wagerAmount,
                 @PayoutAmount AS payoutAmount,
                 @EventTime AS eventTime
     FOR JSON PATH, WITHOUT_ARRAY_WRAPPER);
    

  EXEC dbo.pr_PublishToKafkaJava @EventID = 1500,
                                 @PartitionValue = @UserID,
                                 @EventMessage = @msg; 
END

In [ ]:
CREATE OR ALTER PROCEDURE dbo.pr_LogWager @UserID INT,
                                          @GameID INT,
                                          @WagerAmount bigint,
                                          @PayoutAmount bigint,
                                          @EventTime datetime2
AS
BEGIN
  IF(@EventTime IS NULL)
  BEGIN
    SET @EventTime = SYSUTCDATETIME();
  END 

  BEGIN TRY
    BEGIN TRAN
      INSERT INTO dbo.tb_GamePlay(UserID, GameID, WagerAmount, PayoutAmount, EventTime)
      VALUES(@UserID, @GameID, @WagerAmount, @PayoutAmount, @EventTime);
      --do some more tx "stuff" here
    COMMIT TRAN

    EXEC dbo.pr_GenerateAndPublishWagerEvent @UserID = @UserID,
                                              @GameID = @GameID,
                                              @WagerAmount = @WagerAmount,
                                              @PayoutAmount = @PayoutAmount,
                                              @EventTime = @EventTime,
                                              @eventID = 1500;
                              
  END TRY
  BEGIN CATCH
    ROLLBACK TRAN;
  END CATCH

END  
GO

## Run It

Before we run, don't forget to delete and recreate the wagers topic - to clear it out.

In [ ]:
TRUNCATE TABLE dbo.tb_GamePlay;

In [ ]:
SET NOCOUNT ON;
EXECUTE dbo.pr_EmulateGamePlay @Loops = 3;

In [ ]:
SELECT * FROM dbo.tb_GamePlay;